<a href="https://colab.research.google.com/github/lpedraoss/AlgorithmsUN2023II/blob/main/Group_8_lab7_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Stable Marriage Problem**


In [17]:
# Python3 program for the stable marriage problem

# Number of Men or Women
N = 4

# This function returns true if
# woman 'w' prefers man 'm1' over man 'm'
def wPrefersM1OverM(prefer, w, m, m1):

	# Check if w prefers m over her
	# current engagement m1
	for i in range(N):

		# If m1 comes before m in the list of w,
		# then w prefers her current engagement,
		# don't do anything
		if prefer[w][i] == m1:
			return True

		# If m comes before m1 in w's list,
		# then free her current engagement
		# and engage her with m
		if prefer[w][i] == m:
			return False

# Prints stable matching for N boys and N girls.
# Boys are numbered as 0 to N-1.
# Girls are numbered as N to 2N-1.
def stableMarriage(prefer):

	# Stores the partner of women. This is our output
	# array that stores passing information.
	# The value of wPartner[i] indicates the partner
	# assigned to woman N+i. Note that the woman numbers
	# between N and 2*N-1. The value -1 indicates
	# that the (N+i)'th woman is free
	wPartner = [-1 for i in range(N)]

	# An array to store the availability of men.
	# If mFree[i] is false, then man 'i' is free,
	# otherwise, he is engaged.
	mFree = [False for i in range(N)]

	freeCount = N

	# While there are free men
	while freeCount > 0:

		# Pick the first free man (we could pick any)
		m = 0
		while m < N:
			if not mFree[m]:
				break
			m += 1

		# One by one go to all women according to
		# m's preferences. Here m is the picked free man
		i = 0
		while i < N and not mFree[m]:
			w = prefer[m][i]

			# The woman of preference is free,
			# w and m become partners (Note that
			# the partnership may be changed later).
			# So we can say they are engaged not married
			if wPartner[w - N] == -1:
				wPartner[w - N] = m
				mFree[m] = True
				freeCount -= 1

			else:

				# If w is not free
				# Find the current engagement of w
				m1 = wPartner[w - N]

				# If w prefers m over her current engagement m1,
				# then break the engagement between w and m1 and
				# engage m with w.
				if not wPrefersM1OverM(prefer, w, m, m1):
					wPartner[w - N] = m
					mFree[m] = True
					mFree[m1] = False
			i += 1

			# End of Else
		# End of the for loop that goes
		# to all women in m's list
	# End of main while loop

	# Print solution
	print("Woman ", " Man")
	for i in range(N):
		print(i + N, "\t", wPartner[i])

# Driver Code
prefer = ((7, 5, 6, 4), (5, 4, 6, 7),
          (4, 5, 6, 7), (4, 5, 6, 7),
          (0, 1, 2, 3), (0, 1, 2, 3),
          (0, 1, 2, 3), (0, 1, 2, 3))

stableMarriage(prefer)



Woman   Man
4 	 2
5 	 1
6 	 3
7 	 0


The Gale-Shapley algorithm in Python

David Gale and Lloyd Shapley proved that in cases with when two sets are equal there always a way to create stable pairs. I recommend reading the original paper[2] to be familiar with the elegant prove they provided.

There are different implementations of the Gale-Shapley algorithm in Python [3, 4, 5], let’s create this algorithm closer to pseudocode solution without going into complicated OOP.

In a pseudocode solution looks like this [6]:

function stableMatching {
  
    Initialize all m ∈ M and w ∈ W to free
    while ∃ free man m who still has a woman w to propose to {
       w = first woman on m’s list to whom m has not yet proposed
       if w is free
         (m, w) become engaged
       else some pair (m', w) already exists
         if w prefers m to m'
            m' becomes free
           (m, w) become engaged
         else
           (m', w) remain engaged
    }
}

In [3]:
%pylab inline
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


Define men and women data frames

In [9]:
man_list = ['a', 'b', 'c', 'd']
women_list = ['A', 'B', 'C', 'D']
women_df = pd.DataFrame({'A': [3,4,2,1], 'B': [3,1,4,2], 'C':[2,3,4,1], 'D':[3,2,1,4]})
women_df.index = man_list
man_df = pd.DataFrame({'A': [1,1,2,4], 'B': [2,4,1,2], 'C':[3,3,3,3], 'D':[4,2,4,1]})
man_df.index = man_list

Gale-Shapley algorithm

In [10]:
# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}
# waiting list of men that were able to create pair on each iteration
waiting_list = []
# dict to store created pairs
proposals = {}
# variable to count number of iterations
count = 0

In [11]:
# while not all men have pairs
while len(waiting_list)<len(man_list):
    # man makes proposals
    for man in man_list:
        if man not in waiting_list:
            # each man make proposal to the top women from it's list
            women = women_available[man]
            best_choice = man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin()
            proposals[(man, best_choice)]=(man_df.loc[man][best_choice],
                                                 women_df.loc[man][best_choice])
    # if women have more than one proposals
    # she will choose the best option
    overlays = Counter([key[1] for key in proposals.keys()])
    # cycle to choose the best options
    for women in overlays.keys():
        if overlays[women]>1:
            # pairs to drop from proposals
            pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys()
                    if women in pair}.items(),
                   key=lambda x: x[1][1]
                  )[1:]
            # if man was rejected by woman
            # there is no pint for him to make proposal
            # second time to the same woman
            for p_to_drop in pairs_to_drop:
                del proposals[p_to_drop[0]]
                _women = copy(women_available[p_to_drop[0][0]])
                _women.remove(p_to_drop[0][1])
                women_available[p_to_drop[0][0]] = _women
    # man who successfully created pairs must be added to the waiting list
    waiting_list = [man[0] for man in proposals.keys()]
    # update counter
    count+=1


Stable pairs

In [14]:
proposals

{('b', 'D'): (2, 2),
 ('d', 'B'): (2, 2),
 ('c', 'A'): (2, 2),
 ('a', 'C'): (3, 2)}

In [15]:
count

6